# Cofactor Swap Analysis

Cofactor Swap Analysis is performed to identify possibly reactions where swapping between cofactors NADH and NADPH might improve the flux of surfactin. The analysis was only performed with those cofactors, though it is possible to make the analysis using other cofactors as well. This is done due to limitations in time and scope of assignment. 

The analysis was performed on the model with sucrose as the carbon source, since according to literature and analysis, this is the better carbon source for surfactin production. 

The Cofactor Swap Analysis was performed using modules provided by cameo, as seen imported below. 

In [1]:
%run model_sucrose.ipynb #Getting model with sucrose as carbon source for modifications

In [2]:
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH
from cobra import Reaction
from cameo.core.manipulation import swap_cofactors
#Pulling relevant code from cameo. 

## Running analysis

Below is the background function of the analysis as well as the actual code for identifying targets. 

In [3]:
import time
import numpy as np
def cofactor_optimization(model, reaction='HACD5'): #HACD5 was just an example of a reaction - this is switched to the surfactin producing reactions in the next step. 
    # Starting time for code
    start = time.time()
    
    model.solver = "glpk" #Free software package
    
    biomass = model.reactions.BIOMASS_BS_10 #Reactions leading to biomass formation 
    biomass.lower_bound = 0.1 #Possible to change bounds, but does that make sense to do?
    
    demand = getattr(model.reactions, reaction)
    
    model.objective = demand #Definition on what product we are demanding / looking for and setting it as model objective
    
    pyield = product_yield(demand, model.reactions.EX_sucr_e) #Setting the yield of surfactin based on the surcrose used
    swap_opt = CofactorSwapOptimization(model=model, objective_function=pyield, plot=True, cofactor_id_swaps=(['nadp_c', 'nadph_c'], ['nad_c', 'nadh_c']))
    
    result = swap_opt.run(max_evaluations=100, max_size=5) #Running the analysis
    
    print('Total Time: %.1f' %(time.time()-start))
    
    return result

In [4]:
reactions = ['SP_1', 'SP_2', 'SP_3', 'SP_4']
sol = []
for reaction in reactions:
    sol.append(cofactor_optimization(model, reaction=reaction)) #Running solutions for the 4 reactions leading to surfactin - 4 reactions due to 4 differents fatty acids to be used.

Starting optimization at Sun, 28 Nov 2021 17:47:25


HBox()

Finished after 00:00:16
Total Time: 31.1
Starting optimization at Sun, 28 Nov 2021 17:47:56


HBox()

Finished after 00:00:17
Total Time: 33.1
Starting optimization at Sun, 28 Nov 2021 17:48:30


HBox()

Finished after 00:00:17
Total Time: 30.6
Starting optimization at Sun, 28 Nov 2021 17:49:00


HBox()

Finished after 00:00:21
Total Time: 48.0


Every time the optimization is running, different targets are identified. This might be due to there being a large amount of opportunities and having set the max_evaluation limit to 100 (as to limit run time for the code). For the sake of limiting the scope of this analysis, only three of the identified targets will be chosen for further analysis. 

## Targets identified for Cofactor Swap 

The result of the analysis is printed below and display a vast variety of possible targets for Cofactor Swap. 

In [12]:
sol[0] #Printing the solution for SP_1, ie. the 0th of the sol. 

,index,targets,fitness
0,0,"(G6PDH2r, KAS6)",0.492908
1,1,"(OIVD1r, GND)",0.492885
2,2,"(GND,)",0.492867
3,3,"(PDH,)",0.492242
4,4,"(GAPD,)",0.492242
5,7,"(KAS11, KARA1)",0.491380
6,8,"(KARA1,)",0.491378
7,9,"(PGCD,)",0.490483
8,11,"(SULR_1,)",0.490436
9,12,"(MDH,)",0.490419


In [13]:
sol[1] #Printing solution for SP_2

,index,targets,fitness
0,0,"(PDH,)",0.487500
1,1,"(GAPD,)",0.487500
2,4,"(TRDR, DHDPRy, IPMD)",0.486423
3,5,"(MDH,)",0.486119
4,6,"(PGCD, DPR)",0.486118
5,7,"(IPMD,)",0.486062
6,8,"(SULR_1,)",0.485778
7,12,"(PGCD,)",0.485484
8,13,"(TRDR, OIVD2)",0.485162
9,14,"(TRDR,)",0.485161


In [14]:
sol[2] #Printing solution for SP_3

,index,targets,fitness
0,0,"(GND, ACTD2)",0.486833
1,1,"(GND,)",0.486828
2,2,"(PDH,)",0.486459
3,5,"(GAPD,)",0.486459
4,6,"(G6PDH2r,)",0.485812
5,7,"(KARA1,)",0.485595
6,8,"(PGCD, DHDPRy)",0.484791
7,9,"(PGCD,)",0.484750
8,10,"(SULR_1,)",0.484706
9,11,"(IPMD,)",0.484693


In [15]:
sol[3] #Printing solution for SP_4

,index,targets,fitness
0,0,"(ACOAD3, MDH, GND)",0.489801
1,1,"(GND, ALAD_L)",0.488481
2,2,"(GND,)",0.488382
3,3,"(PDH,)",0.488205
4,6,"(GAPD,)",0.488205
5,9,"(P5CR, KARA1)",0.487308
6,11,"(MDH,)",0.486580
7,12,"(PGCD, PPND)",0.486452
8,13,"(PGCD,)",0.486442
9,14,"(SULR_1,)",0.486398


To identify targets of Cofactor Swap, that are present for all 4 reactions, unique values and then counts of these unique values are summed up to see if any occurs 4 times, ie. in all reactions. 

In [16]:
#Making sure each target columns only has unique values to be able to determine if any reaction occurs as target for cofactor swap for all reactions.
for s in sol:
    _, cts = np.unique(s.data_frame.targets, return_counts=True)
    print(cts)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [17]:
#Adding all targets together in one array and identifying targets that occur 4 times.
all_targets = np.concatenate([s.data_frame.targets.values for s in sol])
_, cts = np.unique(all_targets, return_counts=True)
idx = np.where(cts==4)[0]

In [18]:
all_targets[idx] #Getting index for targets that is shared by all 4 reactions. 

array([('KAS4',), ('KAS13',), ('KAS3',), ('KARA2',), ('OIVD2',),
       ('GND', 'ACTD2')], dtype=object)

The above targets were identified to be present for all 4 reactions leading to surfactin.

From these the following three targets are chosen for further analysis. 
- KAS3 
- GND
- ACTD2

### Swapping cofactor and calculating flux for KAS3

The reaction for KAS3 is printed to determine which swap is going to happen, ie. NADPH to NADH or vice versa. 
After the print, the surfactin flux of the unchanged model is printed together with the flux of the model with the cofactors swapped. This is done for all three reactions. 

In [19]:
model.reactions.KAS3 #Checking which cofactor needs to be swapped - NADPH -> NADH or NADH -> NADPH

Reaction identifier,KAS3
Name,B-ketoacyl synthetase (Iso-C15:0)
Memory address,0x02f7749b7670
Stoichiometry,14.0 h_c + ivcoa_c + 5.0 malcoa_c + 10.0 nadph_c --> 5.0 co2_c + 6.0 coa_c + fa3_c + 4.0 h2o_c + 10.0 nadp_c 14.0 H+ + Isovaleryl-CoA + 5.0 Malonyl CoA C24H33N7O19P3S + 10.0 Nicotinamide adenine dinucleotide phosphate - reduced --> 5.0 CO2 CO2 + 6.0 Coenzyme A + Fatty acid (Iso-C15:0) + 4.0 H2O H2O + 10.0...
GPR,( BSU15900 and BSU11340 and BSU15910 and BSU10170 and BSU08650 and BSU36370 ) or ( BSU15900 and...
Lower bound,0.0
Upper bound,999999.0


In [20]:
with model:
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)
    swap_cofactors(reaction = model.reactions.KAS3, model=model, swap_pairs=([model.metabolites.nad_c, model.metabolites.nadh_c], [model.metabolites.nadp_c, model.metabolites.nadph_c]), inplace=True)
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)
    
#Swapping out the cofactors to see how the flux improves. 

0.2999253714285699
0.29992537142857134


A very, very minor change in surfactin flux is seen. 
This would most likely not be a good target for actually performing cofactor swap, since the work put into changing cofactors would not be outweighted of the result. 

### Swapping cofactor and calculating flux for GND

In [22]:
model.reactions.GND

Reaction identifier,GND
Name,Phosphogluconate dehydrogenase
Memory address,0x02f774a2df70
Stoichiometry,6pgc_c + nadp_c <=> co2_c + nadph_c + ru5p__D_c 6-Phospho-D-gluconate + Nicotinamide adenine dinucleotide phosphate <=> CO2 CO2 + Nicotinamide adenine dinucleotide phosphate - reduced + D-Ribulose 5-phosphate
GPR,BSU23860
Lower bound,-999999.0
Upper bound,999999.0


In [27]:
with model:
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)
    swap_cofactors(reaction = model.reactions.GND, model=model, swap_pairs=([model.metabolites.nadp_c, model.metabolites.nadph_c], [model.metabolites.nad_c, model.metabolites.nadh_c]), inplace=True)
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)

0.29992537142856895
0.2999253714285733


Once again, a very small increase in flux of surfactin is observed. This, as with KAS3, is probably not a worthy target for performing Cofactor Swap in wet lab. 

### Swapping cofactor and calculating flux for ACTD2

In [28]:
model.reactions.ACTD2

Reaction identifier,ACTD2
Name,Acetoin dehydrogenase
Memory address,0x02f77420faf0
Stoichiometry,actn__R_c + coa_c + nad_c --> acald_c + accoa_c + h_c + nadh_c R Acetoin C4H8O2 + Coenzyme A + Nicotinamide adenine dinucleotide --> Acetaldehyde + Acetyl-CoA + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,( BSU29690 and BSU29700 and BSU29710 ) or ( BSU08060 and BSU08070 and BSU08080 and BSU08090 )
Lower bound,0.0
Upper bound,999999.0


In [29]:
with model:
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)
    swap_cofactors(reaction = model.reactions.ACTD2, model=model, swap_pairs=([model.metabolites.nad_c, model.metabolites.nadh_c], [model.metabolites.nadp_c, model.metabolites.nadph_c]), inplace=True)
    model.optimize()
    print(model.reactions.DM_surfactin_c.flux)

0.29992537142856895
0.2999253714285712


Sadly, even the last reaction does not give a large increase in flux when swapping cofactors. 


## Conclusion of analysis

In conclusion, it seems the analysis is functional and are able to identify a lot of targets for Cofactor Swap.

Swapping cofactors for some of the targets identified, and shared by all reactions, resulting in very low increases of flux of surfactin. This might be a limitation of the model (or the code) or simply the fact that the changes really does not do much. 


The results were as following:

| Reaction      | Flux before swap    | Flux after swap   |
| :------------- | :----------: | -----------: |
| KAS3   | 0.2999253714285699 | 0.29992537142857134 |
| GND    | 0.29992537142856895 | 0.2999253714285733 |
| ACTD2  | 0.29992537142856895 | 0.2999253714285712 |

Seemingly, the reaction with highest increase in flux of surfactin, is the GND. Though, the increase is so small, doing the swap in wet lab might not yield useful results. 

### References
The following reference was given for the Cofactor Swap analysis module.
1) King, Z. and Feist, A., 2014. Optimal cofactor swapping can increase the theoretical yield for chemical production in Escherichia coli and Saccharomyces cerevisiae. Metabolic Engineering, 24, pp.117-128.